In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))
print(torch.version.cuda)

PyTorch has version 2.0.0+cu117
11.7


In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import random
import string
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric.transforms as T

from plotly import graph_objs as go
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.model_selection import ParameterGrid
from torch_geometric.data import Data, download_url, extract_gz
from torch_geometric.nn import GAE, GCNConv, VGAE
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import train_test_split_edges, negative_sampling, degree

In [3]:
# set the seeds
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [ ]:
wget https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz

In [8]:
url = 'https://dgl-data.s3-us-west-2.amazonaws.com/dataset/DRKG/drkg.tar.gz'
extract_gz(download_url(url, '.'), '.')



Extracting ./drkg.tar.gz


In [20]:
!tar -xf drkg.gz

tar: drkg.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [19]:
data_path = "/home/chirayu.tripathi/drug_repurpose/relation_glossary.tsv"
df_relation = pd.read_csv(data_path, sep="\t")
df_relation

,Relation-name,Data-source,Connected entity-types,Interaction-type,Description,Reference for the description
0,DGIDB::ACTIVATOR::Gene:Compound,DGIDB,Compound:Gene,activation,An activator interaction is when a drug activa...,http://www.dgidb.org/getting_started
1,DGIDB::AGONIST::Gene:Compound,DGIDB,Compound:Gene,agonism,An agonist interaction occurs when a drug bind...,http://www.dgidb.org/getting_started
2,DGIDB::ALLOSTERIC MODULATOR::Gene:Compound,DGIDB,Compound:Gene,allosteric modulation,An allosteric modulator interaction occurs whe...,http://www.dgidb.org/getting_started
3,DGIDB::ANTAGONIST::Gene:Compound,DGIDB,Compound:Gene,antagonism,An antagonist interaction occurs when a drug b...,http://www.dgidb.org/getting_started
4,DGIDB::ANTIBODY::Gene:Compound,DGIDB,Compound:Gene,antibody,An antibody interaction occurs when an antibod...,http://www.dgidb.org/getting_started
...,...,...,...,...,...,...
102,bioarx::Covid2_acc_host_gene::Disease:Gene,BIBLIOGRAPHY,Disease:Gene,interaction,"Interactions between 27 viral proteins, and ...",NaN
103,bioarx::DrugHumGen:Compound:Gene,BIBLIOGRAPHY,Compound:Gene,interaction,NaN,NaN
104,bioarx::DrugVirGen:Compound:Gene,BIBLIOGRAPHY,Compound:Gene,interaction,NaN,NaN
105,bioarx::HumGenHumGen:Gene:Gene,BIBLIOGRAPHY,Gene:Gene,interaction,Protein-protein interaction,NaN


In [20]:
df_relation['Connected entity-types'].unique()

array(['Compound:Gene', 'Compound:Compound', 'ATC:Compound',
       'Compound:Disease', 'Gene:Gene', 'Disease:Gene', 'Gene:Tax',
       'Anatomy:Gene', 'Compound:Side Effect', 'Anatomy:Disease',
       'Disease:Symptom', 'Diisease:Disease', 'Biological Process:Gene',
       'Cellular Component:Gene', 'Gene:Molecular Function',
       'Gene:Pathway', 'Compound:Pharmacologic Class'], dtype=object)

In [21]:
# df['Connected entity-types'].unique()

In [6]:
df[df['Connected entity-types']=='Disease:Gene']

,Relation-name,Data-source,Connected entity-types,Interaction-type,Description,Reference for the description
24,GNBR::D::Gene:Disease,GNBR,Disease:Gene,drug targets,NaN,NaN
30,GNBR::G::Gene:Disease,GNBR,Disease:Gene,promotes progression,NaN,NaN
34,GNBR::J::Gene:Disease,GNBR,Disease:Gene,role in pathogenesis,NaN,NaN
36,GNBR::L::Gene:Disease,GNBR,Disease:Gene,improper regulation linked to disease,NaN,NaN
37,GNBR::Md::Gene:Disease,GNBR,Disease:Gene,biomarkers (diagnostic),NaN,NaN
47,GNBR::Te::Gene:Disease,GNBR,Disease:Gene,possible therapeutic effect,NaN,NaN
48,GNBR::U::Gene:Disease,GNBR,Disease:Gene,causal mutations,NaN,NaN
49,GNBR::Ud::Gene:Disease,GNBR,Disease:Gene,mutations affecting disease course,NaN,NaN
52,GNBR::X::Gene:Disease,GNBR,Disease:Gene,overexpression in disease,NaN,NaN
53,GNBR::Y::Gene:Disease,GNBR,Disease:Gene,polymorphisms alter risk,NaN,NaN


In [4]:
data_path = "/home/chirayu.tripathi/drug_repurpose/drkg.tsv"
df = pd.read_csv(data_path, sep="\t")
# print(df.head(), '\n')

In [5]:
df['bioarx::HumGenHumGen:Gene:Gene'].unique()

array(['bioarx::HumGenHumGen:Gene:Gene', 'bioarx::VirGenHumGen:Gene:Gene',
       'bioarx::DrugVirGen:Compound:Gene',
       'bioarx::DrugHumGen:Compound:Gene',
       'bioarx::Covid2_acc_host_gene::Disease:Gene',
       'bioarx::Coronavirus_ass_host_gene::Disease:Gene',
       'DGIDB::INHIBITOR::Gene:Compound',
       'DGIDB::ANTAGONIST::Gene:Compound', 'DGIDB::OTHER::Gene:Compound',
       'DGIDB::AGONIST::Gene:Compound', 'DGIDB::BINDER::Gene:Compound',
       'DGIDB::MODULATOR::Gene:Compound', 'DGIDB::BLOCKER::Gene:Compound',
       'DGIDB::CHANNEL BLOCKER::Gene:Compound',
       'DGIDB::ANTIBODY::Gene:Compound',
       'DGIDB::POSITIVE ALLOSTERIC MODULATOR::Gene:Compound',
       'DGIDB::ALLOSTERIC MODULATOR::Gene:Compound',
       'DGIDB::ACTIVATOR::Gene:Compound',
       'DGIDB::PARTIAL AGONIST::Gene:Compound',
       'DRUGBANK::x-atc::Compound:Atc',
       'DRUGBANK::ddi-interactor-in::Compound:Compound',
       'DRUGBANK::target::Compound:Gene',
       'DRUGBANK::enzyme::Compou

In [6]:
def occur(x,count):
    if 'bioarx::HumGenHumGen:' in x:
        return x[len('bioarx::HumGenHumGen:'):]
    if 'bioarx::VirGenHumGen:' in x:
        return x[len('bioarx::VirGenHumGen:'):]
    if 'bioarx::DrugVirGen:' in x:
        return x[len('bioarx::DrugVirGen:'):]
    if 'bioarx::DrugHumGen:' in x:
        return x[len('bioarx::DrugHumGen:'):]
    if x.count('::')<count:
        return x
    if count==0:
        return x
    else:
        x = x[x.find('::')+2:]
        return occur(x,count-1)
    

In [70]:
st = 'bioarx::HumGenHumGen:Gene:Gene'
occur(st,2)

'Gene:Gene'

In [7]:
df[(df['bioarx::HumGenHumGen:Gene:Gene'].str.contains('Compound:Gene'))]
df['category'] = df['bioarx::HumGenHumGen:Gene:Gene'].apply(lambda x: occur(x,2))

In [8]:
df

,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2157.1,category
0,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::5264,Gene:Gene
1,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::2158,Gene:Gene
2,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::3309,Gene:Gene
3,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::28912,Gene:Gene
4,Gene::2157,bioarx::HumGenHumGen:Gene:Gene,Gene::811,Gene:Gene
...,...,...,...,...
5874255,Gene::29099,STRING::REACTION::Gene:Gene,Gene::1643,Gene:Gene
5874256,Gene::51645,STRING::REACTION::Gene:Gene,Gene::3183,Gene:Gene
5874257,Gene::865,STRING::CATALYSIS::Gene:Gene,Gene::983,Gene:Gene
5874258,Gene::1066,STRING::BINDING::Gene:Gene,Gene::7365,Gene:Gene


In [9]:
nodes = df['category'].unique()

In [10]:
li = []
for i in nodes:
    li = li+i.split(':')

In [11]:
nodes_category = set(li)

In [12]:
nodes_category

{'Anatomy',
 'Atc',
 'Biological Process',
 'Cellular Component',
 'Compound',
 'Disease',
 'Gene',
 'Molecular Function',
 'Pathway',
 'Pharmacologic Class',
 'Side Effect',
 'Symptom',
 'Tax'}

In [13]:
# sub_graph='nan'
# sub_graph

In [14]:
combination_graph = pd.DataFrame({'source_node':[],'destination_node':[],'category':[]})
combination_graph['source_node'] = df['Gene::2157']
combination_graph['destination_node'] = df['Gene::2157.1']
combination_graph['category'] = df['category']

In [75]:
combination_graph['category'].unique()

array(['Gene:Gene', 'Compound:Gene', 'Disease:Gene', 'Gene:Compound',
       'Compound:Atc', 'Compound:Compound', 'Compound:Disease',
       'Gene:Disease', 'Gene:Tax', 'Gene:Biological Process',
       'Disease:Symptom', 'Disease:Anatomy', 'Disease:Disease',
       'Anatomy:Gene', 'Gene:Molecular Function',
       'Pharmacologic Class:Compound', 'Gene:Cellular Component',
       'Gene:Pathway', 'Compound:Side Effect'], dtype=object)

In [16]:
node_count = {}
for i in nodes_category:
    seta = list(combination_graph[(combination_graph['source_node'].str.contains(i))]['source_node'].unique())
    setb = list(combination_graph[(combination_graph['destination_node'].str.contains(i))]['destination_node'].unique())
    length = len(set(seta+setb))
    node_count[i] = length
# combination_graph[(combination_graph['destination_node'].str.contains('Gene'))]['destination_node'].unique()

In [17]:
node_count

{'Cellular Component': 1391,
 'Disease': 5103,
 'Pathway': 1822,
 'Compound': 24313,
 'Atc': 4048,
 'Anatomy': 400,
 'Pharmacologic Class': 345,
 'Gene': 39220,
 'Symptom': 415,
 'Side Effect': 5701,
 'Tax': 215,
 'Biological Process': 11381,
 'Molecular Function': 2884}

In [18]:
sumi = 0
for i in node_count:
    sumi+=node_count[i]
print(sumi)

97238


In [19]:
# len(sub_graph['Disease_id'].unique()),len(sub_graph['Compound_id'].unique())

In [20]:
def load_node_mapping(df, source_node, destination_node, nodes_category, node_count):

#   df = pd.read_csv(datafile_path, index_col=index_col, sep="\t")
#   seta = list(df[(df[source_node].str.contains(category))][source_node].unique())
#   setb = list(df[(df[destination_node].str.contains(category))][destination_node].unique())
#   nodes = list(set(seta+setb))
#   mapping = {index_id: i + offset for i, index_id in enumerate(nodes)}
    mapping = {}
    offset = 0
    for i in nodes_category:
        seta = list(df[(df[source_node].str.contains(i))][source_node].unique())
        setb = list(df[(df[destination_node].str.contains(i))][destination_node].unique())
        nodes = list(set(seta+setb))
        temp = {index_id: j + offset for j, index_id in enumerate(nodes)}
        offset += node_count[i]
        mapping[i] = temp
#   print(mapping)
    return mapping

def load_edge_list(df, src_col, dst_col, mapping, nodes_category):

#   df = pd.read_csv(datafile_path, sep="\t")
    
#   src_nodes = [src_mapping[index] for index in df[src_col]]
#   dst_nodes = [dst_mapping[index] for index in df[dst_col]]
#   edge_index = torch.tensor([src_nodes, dst_nodes])
    src_nodes = []
    for i in df[src_col]:
        for j in nodes_category:
            if j in i:
                src_nodes.append(mapping[j][i])
                break

            
    dst_nodes = []
    for i in df[dst_col]:
        for j in nodes_category:
            if j in i:
                dst_nodes.append(mapping[j][i])
                break
    print(src_nodes[0:5])
    print(dst_nodes[0:5])
    edge_index = torch.tensor([src_nodes, dst_nodes])
    return edge_index

def initialize_data(df, nodes_category, node_count, num_features=1):


#   dz_col, compound_col = "Disease_id", "Compound_id"
    source_node, destination_node = "source_node", "destination_node"
    mapping = load_node_mapping(df, source_node, destination_node, nodes_category, node_count)
#   compound_mapping = load_node_mapping(df, source_node, destination_node,'Compound', offset=5070)
#   gene_mapping = load_node_mapping(df, source_node, destination_node,'Gene', offset=5070+23311)

  # Get edge index in terms of the integer indeces assigned to the nodes.
    edge_index = load_edge_list(
      df, source_node, destination_node, mapping, nodes_category)

  # Add the reverse direction (aka make it a undirected graph)
    rev_edge_index = load_edge_list(
      df, destination_node,  source_node, mapping, nodes_category)
    sumi = 0
    for i in mapping:
        sumi+=len(mapping[i])
  # Construct a Data object.
    data = Data()
    data.num_nodes = sumi
    data.edge_index = torch.cat((edge_index, rev_edge_index), dim=1)
#   data.edge_index = edge_index
  # pretend we have uniform node features
    data.x = torch.ones((data.num_nodes, num_features))

    return data, mapping

In [21]:
data_object, mapping = initialize_data(combination_graph, nodes_category, node_count)
print(data_object)
# print("Number of Compounds:", len(compound_mapping))
# print("Number of Diseases:", len(dz_mapping))
# print("Number of Genes:", len(gene_mapping))

[44653, 44653, 44653, 44653, 44653]
[63034, 47608, 57367, 47520, 60853]
[63034, 47608, 57367, 47520, 60853]
[44653, 44653, 44653, 44653, 44653]
Data(num_nodes=97238, edge_index=[2, 11748520], x=[97238, 1])


In [22]:
mapping['Gene']['Gene::2157'], mapping['Gene']['Gene::2158']

(44653, 47608)

In [23]:
11748520/2

5874260.0

In [56]:
node_count

{'Cellular Component': 1391,
 'Disease': 5103,
 'Pathway': 1822,
 'Compound': 24313,
 'Atc': 4048,
 'Anatomy': 400,
 'Pharmacologic Class': 345,
 'Gene': 39220,
 'Symptom': 415,
 'Side Effect': 5701,
 'Tax': 215,
 'Biological Process': 11381,
 'Molecular Function': 2884}

In [57]:
for i in mapping:
    print(i,' ',list(mapping[i].values())[0],' ',list(mapping[i].values())[-1])

Cellular Component   0   1390
Disease   1391   6493
Pathway   6494   8315
Compound   8316   32628
Atc   32629   36676
Anatomy   36677   37076
Pharmacologic Class   37077   37421
Gene   37422   76641
Symptom   76642   77056
Side Effect   77057   82757
Tax   82758   82972
Biological Process   82973   94353
Molecular Function   94354   97237


In [26]:
# compound_mapping

In [113]:
# print(compound_mapping)

In [114]:
# compound_mapping
# gene_mapping
print(data_object.edge_index)

tensor([[23012, 14809, 14809,  ..., 53947, 49702, 46265],
        [53846, 38822, 43423,  ..., 19795, 20940,  7501]])


In [115]:
reverse_dz_mapping = {j: i for i,j in dz_mapping.items()}
reverse_gene_mapping = {j: i for i,j in gene_mapping.items()}
reverse_compound_mapping = {j: i for i,j in compound_mapping.items()}

degrees = degree(data_object.edge_index[0]).numpy()
sorted_degrees_i = np.argsort(-1* degrees)

print("Disease Nodes of highest degree")
top_k = 10
num_dz_nodes = 0
for i in sorted_degrees_i:
  if i < 5070:   
    node_id = reverse_dz_mapping[i]
    node_degree = degrees[i]
    print("node_index=" + str(i), "node_degree=" + str(int(node_degree)),
          node_id)
    num_dz_nodes += 1
    if num_dz_nodes >= top_k:
      break

print("\nGene Nodes of highest degree")
num_gene_nodes = 0
for i in sorted_degrees_i:
  if i >= 5070+23311:
    node_id = reverse_gene_mapping[i]
    node_degree = degrees[i]
    print("node_index=" + str(i), "node_degree=" + str(int(node_degree)), node_id)
    num_gene_nodes += 1
    if num_gene_nodes >= top_k:
      break
    
print("\nCompound Nodes of highest degree")
num_gene_nodes = 0
for i in sorted_degrees_i:
  if 5070+23311>i >= 5070:
    node_id = reverse_compound_mapping[i]
    node_degree = degrees[i]
    print("node_index=" + str(i), "node_degree=" + str(int(node_degree)), node_id)
    num_gene_nodes += 1
    if num_gene_nodes >= top_k:
      break

Disease Nodes of highest degree
node_index=1255 node_degree=8156 Disease::MESH:D009369
node_index=1905 node_degree=3428 Disease::MESH:D001943
node_index=4849 node_degree=2679 Disease::MESH:D006528
node_index=1410 node_degree=2677 Disease::MESH:D015179
node_index=4859 node_degree=2551 Disease::MESH:D011471
node_index=3261 node_degree=2463 Disease::MESH:D008545
node_index=2774 node_degree=2400 Disease::MESH:D013274
node_index=2990 node_degree=2139 Disease::MESH:D064420
node_index=4109 node_degree=2034 Disease::MESH:D010190
node_index=4535 node_degree=2007 Disease::MESH:D010051

Gene Nodes of highest degree
node_index=40762 node_degree=2483 Gene::1576
node_index=38590 node_degree=1213 Gene::1565
node_index=45346 node_degree=1166 Gene::5243
node_index=42487 node_degree=1126 Gene::3630
node_index=30086 node_degree=1093 Gene::7124
node_index=57504 node_degree=1041 Gene::1559
node_index=41510 node_degree=987 Gene::213
node_index=29426 node_degree=986 Gene::1544
node_index=47233 node_degree=91

In [28]:
NUM_FEATURES =   20

In [29]:
data_object.x = torch.ones((data_object.num_nodes, NUM_FEATURES))
print("Using dummy embeddings as initial node features.")
print("Number of features set to ", NUM_FEATURES)

Using dummy embeddings as initial node features.
Number of features set to  20


In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
device

device(type='cuda')

In [37]:
transform = T.Compose([
#     T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.15, is_undirected=True,
                      split_labels=True, add_negative_train_samples=True),])

train_dataset, val_dataset, test_dataset = transform(data_object)
print("Train Data:\n", train_dataset)
print("Validation Data:\n", val_dataset)
print("Test Data:\n", test_dataset)

Train Data:
 Data(num_nodes=97238, edge_index=[2, 9404416], x=[97238, 20], pos_edge_label=[4702208], pos_edge_label_index=[2, 4702208], neg_edge_label=[4702208], neg_edge_label_index=[2, 4702208])
Validation Data:
 Data(num_nodes=97238, edge_index=[2, 9404416], x=[97238, 20], pos_edge_label=[293887], pos_edge_label_index=[2, 293887], neg_edge_label=[293887], neg_edge_label_index=[2, 293887])
Test Data:
 Data(num_nodes=97238, edge_index=[2, 9992190], x=[97238, 20], pos_edge_label=[881663], pos_edge_label_index=[2, 881663], neg_edge_label=[881663], neg_edge_label_index=[2, 881663])


In [121]:
def get_mapping(df):
  
#   disease_mapping = [index_id for index_id in enumerate(df['Disease_id'].unique())]
# #   df = pd.read_csv(data_path, index_col="Gene ID", sep="\t")
#   gene_mapping = [index_id[1] for index_id in enumerate(df['Compound_id'].unique())]
  disease_mapping = [i for i in dz_mapping.keys()]
  gen_mapping = [i for i in gene_mapping.keys()]
  comp_mapping = [i for i in compound_mapping.keys()]
  mapping = disease_mapping + gen_mapping + comp_mapping
  return mapping

def visualize_tsne_embeddings(model, data, title, perplexity=30.0,
                              labeled=False, labels=[]):



  model.eval()
  x = data.x
  z = model.encode(x, data.edge_index)
  print('z done')

  ax1, ax2 = zip(*TSNE(n_components=2, learning_rate='auto', perplexity=perplexity,
                       init='random').fit_transform(z.detach().cpu().numpy()))

  fig = px.scatter(x=ax1, y=ax2, color=['r']*5070 + ['g']*23311+['b']*31362,
                   hover_data=[get_mapping(combination_graph)],
                   title=title)

  if labeled:
    for i in labels:
      fig.add_annotation(x=ax1[i], y=ax2[i],
                         text=str(i), showarrow=False)
  fig.show()

def visualize_pca_embeddings(model, data, title, labeled=False, labels=[]):

  model.eval()
  x = data.x
  z = model.encode(x, data.edge_index)

  pca = PCA(n_components=2)
  components = pca.fit_transform(z.detach().cpu().numpy())
  fig = px.scatter(components, x=0, y=1, color=['r']*5070 + ['g']*23311+['b']*31362,
                   hover_data=[get_mapping(combination_graph)], title=title)

  if labeled:
    for i in labels:
      fig.add_annotation(x=components[:,0][i], y=components[:,1][i],
                         text=str(i), showarrow=False)
  fig.show()

In [122]:
def plot_roc_curve(model, data):

  model.eval()

  x = data.x
  z = model.encode(x, data.edge_index)

  pos_preds = model.decode(z, data.pos_edge_label_index, sigmoid=True)
  neg_preds = model.decode(z, data.neg_edge_label_index, sigmoid=True)
  preds = torch.cat([pos_preds, neg_preds], dim=0)
  preds = preds.detach().cpu().numpy()

  labels = torch.cat((data.pos_edge_label, data.neg_edge_label), dim=0)
  labels = labels.detach().cpu().numpy()
  fpr, tpr, thresholds = metrics.roc_curve(labels, preds)

  # Using J-statistic: https://en.wikipedia.org/wiki/Youden%27s_J_statistic
  J = tpr - fpr
  ix = np.argmax(J)
  best_thresh = thresholds[ix]
  print('Best Threshold=%f' % (best_thresh))

  roc_auc = metrics.roc_auc_score(labels, preds)

  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.plot([0, 1], [0, 1],'r--') # diagonal roc curve of a random classifier
  plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best=%0.2f' % best_thresh)
  plt.xlim(0, 1)
  plt.ylim(0, 1)
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.legend(loc = 'lower right')
  plt.title('ROC curve for model predictions')
  plt.show()

In [123]:
def plot_training_stats(title, losses, test_auc, test_ap, train_auc, train_ap):

  fig, ax = plt.subplots()
  ax2 = ax.twinx()

  ax.set_xlabel("Training Epochs")
  ax2.set_ylabel("Performance Metric")
  ax.set_ylabel("Loss")

  plt.title(title)
  p1, = ax.plot(losses, "b-", label="training loss")
  p2, = ax2.plot(test_auc, "r-", label="test AUC")
  p3, = ax2.plot(test_ap, "g-", label="test AP")
  p4, = ax2.plot(train_auc, "o-", label="train AUC")
  p5, = ax2.plot(train_ap, "v-", label="train AP")
  plt.legend(handles=[p1, p2, p3, p4, p5])
  plt.show()

In [65]:
def get_edge_dot_products(data, model, mapping, node_count):

    model.eval()
    x = data.x
    z = model.encode(x, data.edge_index).detach().cpu().numpy()
    print(len(z))
    offset = 0
    embeddings = {}
    for i in mapping:
        embeddings[i] = z[offset:offset+node_count[i], :]
        offset+=node_count[i]
    
    
    
#   dz_z = z[:num_dz_nodes, :]
#   com_z = z[num_dz_nodes:num_dz_nodes+num_com_nodes, :]
#   gene_z = z[num_dz_nodes+num_com_nodes:, :]
#   print(len(dz_z),len(com_z),len(gene_z))
# (23311, 5070, 31362)


    dot_products_dz_com = np.einsum('ai,bi->ab', embeddings['Disease'], embeddings['Compound'])
#   dot_products_com_gene = np.einsum('ai,bi->ab', com_z, gene_z)
#   dot_products_gene_dz = np.einsum('ai,bi->ab', gene_z, dz_z)
    return dot_products_dz_com  


def get_ranked_edges(data_object, model, mapping, node_count):

  # Get dot products
    dz_com = get_edge_dot_products(data_object, model, mapping, node_count)
#   if interaction == 'dz_com':
    return get_ranking(dz_com,node_count['Disease'])
#   elif interaction == 'com_gene':
#         return get_ranking(com_gene,num_com_nodes)
#   else:
#         return get_ranking(gene_dz,num_gene_nodes)
    
def get_ranking(edge_dot_products,nodes):

      num_possible_edges = edge_dot_products.shape[0] * edge_dot_products.shape[1]

      # Get indeces, ranked by dot product in descending order. This is a tuple (indeces[0], indeces[1]).
      ranked_edges = np.unravel_index(np.argsort(-1 * edge_dot_products, axis=None), edge_dot_products.shape)
      assert len(ranked_edges[0]) == num_possible_edges

      # Get the corresponding, ranked edge list and ranked dot products. 
      # we need to add an offset for the gene_node indeces.
      offset = np.array([np.zeros(num_possible_edges, dtype=int), nodes + np.ones(num_possible_edges, dtype=int)]).T
      ranked_edge_list = np.dstack(ranked_edges)[0] + offset
#       print(ranked_edge_list)
      assert ranked_edge_list.shape[0] == num_possible_edges

      # Get the corresponding ranked dot products
      ranked_dot_products = edge_dot_products[ranked_edges]
      assert ranked_dot_products.shape[0] == num_possible_edges

      return ranked_edge_list, ranked_dot_products

# GAE

In [32]:
HIDDEN_SIZE = 200  #@param {type: "integer"}

OUT_CHANNELS = 20  #@param {type: "integer"}

EPOCHS =   40#@param {type: "integer"}

In [33]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_size, out_channels, dropout):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_size,cached=False) # cached only for transductive learning
        self.conv2 = GCNConv(hidden_size, out_channels,cached=False) # cached only for transductive learning
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index):
        x_temp1 = self.conv1(x, edge_index).relu()
        x_temp2 = self.dropout(x_temp1)
        return self.conv2(x_temp2, edge_index)

    
gae_model = GAE(GCNEncoder(NUM_FEATURES, HIDDEN_SIZE, OUT_CHANNELS, 0.5))
gae_model = gae_model.to(device)

In [34]:
gae_model

GAE(
  (encoder): GCNEncoder(
    (conv1): GCNConv(20, 200)
    (conv2): GCNConv(200, 20)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): InnerProductDecoder()
)

In [35]:
def gae_train(train_data, gae_model, optimizer):
    gae_model.train()
    optimizer.zero_grad()
    z = gae_model.encode(train_data.x, train_data.edge_index)
#     print(z[0])
#     print(train_data.pos_edge_label_index.to(device)[0])
    loss = gae_model.recon_loss(z, train_data.pos_edge_label_index.to(device))
#     print(loss)
    loss.backward(retain_graph=True)
    optimizer.step()
    return float(loss)

@torch.no_grad()
def gae_test(test_data, gae_model):
    gae_model.eval()
    z = gae_model.encode(test_data.x, test_data.edge_index)
    return gae_model.test(z, test_data.pos_edge_label_index, test_data.neg_edge_label_index)

In [36]:
# visualize_tsne_embeddings(gae_model, train_dataset, 'Untrained GAE: train set embeddings t-SNE', labeled=True, labels=[40, 190, 230, 1830, 260, 110, 280, 1967])

In [60]:
# visualize_pca_embeddings(gae_model, train_dataset, 'Untrained GAE: train set embeddings PCA', labeled=True, labels=[40, 190, 230, 1830, 260, 110, 280, 1967])

In [46]:

# import timeit

In [38]:
# %%timeit
# %%time
losses = []
test_auc = []
test_ap = []
train_aucs = []
train_aps = []


optimizer = torch.optim.Adam(gae_model.parameters(), lr=0.01)
# def tr():
for epoch in range(1, 40):
    loss = gae_train(train_dataset, gae_model, optimizer)

    losses.append(loss)
    auc, ap = gae_test(test_dataset, gae_model)
    test_auc.append(auc)
    test_ap.append(ap)

    train_auc, train_ap = gae_test(train_dataset, gae_model)

    train_aucs.append(train_auc)
    train_aps.append(train_ap)

    print('Epoch: {:03d}, test AUC: {:.4f}, test AP: {:.4f}, train AUC: {:.4f}, train AP: {:.4f}, loss:{:.4f}'.format(epoch, auc, ap, train_auc, train_ap, loss))



Epoch: 001, test AUC: 0.9623, test AP: 0.9619, train AUC: 0.9600, train AP: 0.9596, loss:1.3751
Epoch: 002, test AUC: 0.9624, test AP: 0.9619, train AUC: 0.9600, train AP: 0.9597, loss:1.4338
Epoch: 003, test AUC: 0.9624, test AP: 0.9620, train AUC: 0.9601, train AP: 0.9598, loss:1.1325
Epoch: 004, test AUC: 0.9624, test AP: 0.9620, train AUC: 0.9600, train AP: 0.9598, loss:1.1138
Epoch: 005, test AUC: 0.9624, test AP: 0.9621, train AUC: 0.9600, train AP: 0.9599, loss:1.0951
Epoch: 006, test AUC: 0.9624, test AP: 0.9621, train AUC: 0.9600, train AP: 0.9599, loss:1.0718
Epoch: 007, test AUC: 0.9624, test AP: 0.9621, train AUC: 0.9600, train AP: 0.9599, loss:1.0704
Epoch: 008, test AUC: 0.9624, test AP: 0.9621, train AUC: 0.9600, train AP: 0.9599, loss:1.0680
Epoch: 009, test AUC: 0.9623, test AP: 0.9622, train AUC: 0.9599, train AP: 0.9599, loss:1.0722
Epoch: 010, test AUC: 0.9623, test AP: 0.9622, train AUC: 0.9599, train AP: 0.9600, loss:1.0694
Epoch: 011, test AUC: 0.9623, test AP: 0

In [44]:
%timeit -r 1 -n 1 tr()

In [64]:
# gae_model.eval()
# x = data_object.x
# z = gae_model.encode(x, data_object.edge_index).detach().cpu().numpy()
# print('z done')

In [104]:
# visualize_tsne_embeddings(
#     gae_model, train_dataset, title='Trained GAE: train set embeddings',
#     perplexity=5, labeled=True, labels=[40, 190, 230, 1830, 260, 110, 280, 1967])

In [39]:
data_object_to_analyze = data_object    
model_to_analyze = gae_model            

print("DataObject:\n", data_object_to_analyze)
print("\nModel:\n", model_to_analyze)

DataObject:
 Data(num_nodes=97238, edge_index=[2, 11748520], x=[97238, 20])

Model:
 GAE(
  (encoder): GCNEncoder(
    (conv1): GCNConv(20, 200)
    (conv2): GCNConv(200, 20)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): InnerProductDecoder()
)


In [148]:
def get_edge_dot_products(data, model, mapping, node_count):

    model.eval()
    x = data.x
    z = model.encode(x, data.edge_index).detach().cpu().numpy()
    print(len(z))
    offset = 0
    embeddings = {}
    for i in mapping:
        
        embeddings[i] = z[offset:offset+node_count[i], :]
#         print(i,' ', len(embeddings[i]))
        offset+=node_count[i]
    
    
    
#   dz_z = z[:num_dz_nodes, :]
#   com_z = z[num_dz_nodes:num_dz_nodes+num_com_nodes, :]
#   gene_z = z[num_dz_nodes+num_com_nodes:, :]
#   print(len(dz_z),len(com_z),len(gene_z))
# (23311, 5070, 31362)


    dot_products_dz_com = np.einsum('ai,bi->ab', embeddings['Disease'], embeddings['Compound'])
#   dot_products_com_gene = np.einsum('ai,bi->ab', com_z, gene_z)
#   dot_products_gene_dz = np.einsum('ai,bi->ab', gene_z, dz_z)
    return dot_products_dz_com  


def get_ranked_edges(data_object, model, mapping, node_count):

  # Get dot products
    dz_com = get_edge_dot_products(data_object, model, mapping, node_count)
#   if interaction == 'dz_com':
    return get_ranking(dz_com,node_count['Cellular Component'],node_count['Disease']+node_count['Cellular Component']+node_count['Pathway'])
#   elif interaction == 'com_gene':
#         return get_ranking(com_gene,num_com_nodes)
#   else:
#         return get_ranking(gene_dz,num_gene_nodes)
    
def get_ranking(edge_dot_products,disease_pad,nodes):

    num_possible_edges = edge_dot_products.shape[0] * edge_dot_products.shape[1]
    print(edge_dot_products.shape[0],edge_dot_products.shape[1])
    print(edge_dot_products)
    # Get indeces, ranked by dot product in descending order. This is a tuple (indeces[0], indeces[1]).
    ranked_edges = np.unravel_index(np.argsort(-1 * edge_dot_products, axis=None), edge_dot_products.shape)
    assert len(ranked_edges[0]) == num_possible_edges
    print(ranked_edges)
#     print(min(list(ranked_edges[0])))
    # Get the corresponding, ranked edge list and ranked dot products. 
    # we need to add an offset for the gene_node indeces.
#     offset = np.array([np.zeros(num_possible_edges, dtype=int), nodes + np.ones(num_possible_edges, dtype=int)]).T
    offset = np.array([disease_pad+np.zeros(num_possible_edges, dtype=int), nodes + np.zeros(num_possible_edges, dtype=int)]).T
#     off1 = np.array([np.zeros(num_possible_edges, dtype=int), nodes + np.ones(num_possible_edges, dtype=int)])
    print(np.dstack(ranked_edges)[0])
    ranked_edge_list = np.dstack(ranked_edges)[0] + offset
    #       print(ranked_edge_list)
    print(ranked_edge_list)
    assert ranked_edge_list.shape[0] == num_possible_edges
    maxi = 0
    mini2 = 0

    for i,j in ranked_edge_list:
        if i>maxi:
            maxi = i
        if j>mini2:
            mini2 = j
    print(maxi,mini2)

    # Get the corresponding ranked dot products
    ranked_dot_products = edge_dot_products[ranked_edges]
    assert ranked_dot_products.shape[0] == num_possible_edges

    return ranked_edge_list, ranked_dot_products

In [149]:
mapping.keys()

dict_keys(['Cellular Component', 'Disease', 'Pathway', 'Compound', 'Atc', 'Anatomy', 'Pharmacologic Class', 'Gene', 'Symptom', 'Side Effect', 'Tax', 'Biological Process', 'Molecular Function'])

In [150]:
ranked_edge_list, ranked_dot_products = get_ranked_edges(data_object_to_analyze, model_to_analyze, mapping, node_count)


97238
5103 24313
[[ 0.08149116 -0.01615149 -0.02593692 ... -0.01814115  0.5453022
  -0.08829944]
 [-0.07778919  0.01783312  0.02713065 ...  0.01965071 -0.53087616
   0.08828311]
 [-0.02801382  0.00723565  0.0104073  ...  0.0077804  -0.19399865
   0.03301234]
 ...
 [-0.01687022  0.00476088  0.00722606 ...  0.00535359 -0.1208465
   0.0209742 ]
 [-0.05132126  0.0121572   0.01846997 ...  0.01343495 -0.35267827
   0.05903341]
 [ 0.10146434 -0.02040736 -0.03256564 ... -0.02286267  0.68013704
  -0.11041456]]
(array([4455, 4455, 4455, ..., 4455, 4455, 4455]), array([10985,  9528,  2498, ..., 16078, 22838, 11228]))
[[ 4455 10985]
 [ 4455  9528]
 [ 4455  2498]
 ...
 [ 4455 16078]
 [ 4455 22838]
 [ 4455 11228]]
[[ 5846 19301]
 [ 5846 17844]
 [ 5846 10814]
 ...
 [ 5846 24394]
 [ 5846 31154]
 [ 5846 19544]]
6493 32628


In [128]:
node_count

{'Cellular Component': 1391,
 'Disease': 5103,
 'Pathway': 1822,
 'Compound': 24313,
 'Atc': 4048,
 'Anatomy': 400,
 'Pharmacologic Class': 345,
 'Gene': 39220,
 'Symptom': 415,
 'Side Effect': 5701,
 'Tax': 215,
 'Biological Process': 11381,
 'Molecular Function': 2884}

In [129]:
for i in mapping:
    print(i,' ',list(mapping[i].values())[0],' ',list(mapping[i].values())[-1])

Cellular Component   0   1390
Disease   1391   6493
Pathway   6494   8315
Compound   8316   32628
Atc   32629   36676
Anatomy   36677   37076
Pharmacologic Class   37077   37421
Gene   37422   76641
Symptom   76642   77056
Side Effect   77057   82757
Tax   82758   82972
Biological Process   82973   94353
Molecular Function   94354   97237


In [100]:
1391+5103+1822
10985+8316

19301

In [130]:
select_gene_substrings = ''
# select_disease_substrings = ['D001714']
select_disease_substrings = ["D018805"]
# (23311, 5070, 31362)

In [131]:
mapping.keys()

dict_keys(['Cellular Component', 'Disease', 'Pathway', 'Compound', 'Atc', 'Anatomy', 'Pharmacologic Class', 'Gene', 'Symptom', 'Side Effect', 'Tax', 'Biological Process', 'Molecular Function'])

In [151]:
# mapping['Disease']
dz_mapping = mapping['Disease']
compound_mapping = mapping['Compound']

In [152]:
# print(compound_mapping)

In [153]:
# Filter data frame by the selected disease terms, and get their node ids.
if select_disease_substrings:
  dz_regex = "(?i)" + "|".join(select_disease_substrings)
  query_df = combination_graph[(combination_graph['source_node'].str.contains(dz_regex)) | (combination_graph['destination_node'].str.contains(dz_regex))]
#   print(query_df)
  query_dz_nodes = [dz_mapping[dz_id] for dz_id in query_df['source_node'].drop_duplicates().tolist() if 'Disease' in dz_id]
  query_dz_nodes2 = [dz_mapping[dz_id] for dz_id in query_df['destination_node'].drop_duplicates().tolist() if 'Disease' in dz_id]
#   print(query_dz_nodes)
  query_dz_nodes = query_dz_nodes + query_dz_nodes2
#   for i in query_df.iterrows():
        
  # print("\nQueried Disease Nodes:\n", query_df['Disease Name'])
else:
  query_dz_nodes = None

# Filter data frame by the selected gene terms, and get their node ids.
if select_gene_substrings:
  gene_regex = "(?i)" + "|".join(select_gene_substrings)
  query_df = combination_graph[combination_graph['Compound_id'].notna()]
  query_df = query_df[(query_df['source_node'].str.contains(gene_regex)) | (query_df['destination_node'].str.contains(gene_regex))]
  query_gene_nodes = [gene_mapping[gene_id] for gene_id in query_df['Compound_id'].drop_duplicates().tolist()]
  # print("\nQueried Gene Nodes:\n", query_df['Gene Name'])
else:
  query_gene_nodes = None

# print(query_dz_nodes)
# Get reverse dz and gene mappings, to print out all needed info.
reverse_dz_mapping = {j: i for i,j in dz_mapping.items()}
# reverse_gene_mapping = {j: i for i,j in gene_mapping.items()}
reverse_compound_mapping = {j: i for i,j in compound_mapping.items()}

print("\nTop Predicted Edges")
top_k = 50
curr_k = 0
for dz_i, gene_i in ranked_edge_list:
  
  if query_dz_nodes and dz_i not in query_dz_nodes: #and gene_i not in reverse_compound_mapping
    continue
  if query_gene_nodes and gene_i not in query_gene_nodes:
    continue
#   print(dz_i)

  dz_id = reverse_dz_mapping[dz_i] 
  gene_id = reverse_compound_mapping[gene_i]
#   dz_description = combination_graph[combination_graph["Disease_id"] == dz_id]["Disease_id"].drop_duplicates().iloc[0]
#   gene_description = combination_graph[combination_graph["Compound_id"] == gene_id]["Compound_id"].drop_duplicates().iloc[0]
  dot_product = ranked_dot_products[curr_k]

#   print('edge=({},{}), \t dotprod={:.2f},\t descriptions=({},{},{},{})'.format(dz_i, gene_i, dot_product, dz_description, gene_description,dz_id,gene_id))
  print('edge=({},{}), \t dotprod={:.2f},\t descriptions=({},{})'.format(dz_i, gene_i, dot_product,dz_id,gene_id))

  curr_k += 1
  if curr_k > top_k:
    break


Top Predicted Edges
edge=(4841,19301), 	 dotprod=77.73,	 descriptions=(Disease::MESH:D018805,Compound::DB09341)
edge=(4841,17844), 	 dotprod=68.34,	 descriptions=(Disease::MESH:D018805,Compound::DB01373)
edge=(4841,10814), 	 dotprod=63.40,	 descriptions=(Disease::MESH:D018805,Compound::CHEBI:18186)
edge=(4841,24300), 	 dotprod=62.58,	 descriptions=(Disease::MESH:D018805,Compound::CHEBI:33704)
edge=(4841,17768), 	 dotprod=62.39,	 descriptions=(Disease::MESH:D018805,Compound::DB00898)
edge=(4841,20120), 	 dotprod=58.38,	 descriptions=(Disease::MESH:D018805,Compound::DB01234)
edge=(4841,11793), 	 dotprod=56.07,	 descriptions=(Disease::MESH:D018805,Compound::DB09140)
edge=(4841,24034), 	 dotprod=53.36,	 descriptions=(Disease::MESH:D018805,Compound::MESH:D004967)
edge=(4841,13140), 	 dotprod=51.60,	 descriptions=(Disease::MESH:D018805,Compound::MESH:D009569)
edge=(4841,27019), 	 dotprod=50.82,	 descriptions=(Disease::MESH:D018805,Compound::DB02527)
edge=(4841,9366), 	 dotprod=50.34,	 descr

In [70]:
# Filter data frame by the selected disease terms, and get their node ids.
if select_disease_substrings:
  dz_regex = "(?i)" + "|".join(select_disease_substrings)
  query_df = combination_graph[(combination_graph['source_node'].str.contains(dz_regex)) | (combination_graph['destination_node'].str.contains(dz_regex))]
#   print(query_df)
  query_dz_nodes = [dz_mapping[dz_id] for dz_id in query_df['source_node'].drop_duplicates().tolist() if 'Disease' in dz_id]
  query_dz_nodes2 = [dz_mapping[dz_id] for dz_id in query_df['destination_node'].drop_duplicates().tolist() if 'Disease' in dz_id]
#   print(query_dz_nodes)
  query_dz_nodes = query_dz_nodes + query_dz_nodes2
#   for i in query_df.iterrows():
        
  # print("\nQueried Disease Nodes:\n", query_df['Disease Name'])
else:
  query_dz_nodes = None

# Filter data frame by the selected gene terms, and get their node ids.
if select_gene_substrings:
  gene_regex = "(?i)" + "|".join(select_gene_substrings)
  query_df = combination_graph[combination_graph['Compound_id'].notna()]
  query_df = query_df[(query_df['source_node'].str.contains(gene_regex)) | (query_df['destination_node'].str.contains(gene_regex))]
  query_gene_nodes = [gene_mapping[gene_id] for gene_id in query_df['Compound_id'].drop_duplicates().tolist()]
  # print("\nQueried Gene Nodes:\n", query_df['Gene Name'])
else:
  query_gene_nodes = None

# print(query_dz_nodes)
# Get reverse dz and gene mappings, to print out all needed info.
reverse_dz_mapping = {j: i for i,j in dz_mapping.items()}
# reverse_gene_mapping = {j: i for i,j in gene_mapping.items()}
reverse_compound_mapping = {j: i for i,j in compound_mapping.items()}

print("\nTop Predicted Edges")
top_k = 50
curr_k = 0
for dz_i, gene_i in ranked_edge_list:
  
  if query_dz_nodes and dz_i not in query_dz_nodes: #and gene_i not in reverse_compound_mapping
    continue
  if query_gene_nodes and gene_i not in query_gene_nodes:
    continue
#   print(dz_i)

  dz_id = reverse_dz_mapping[dz_i] 
  gene_id = reverse_compound_mapping[gene_i]
#   dz_description = combination_graph[combination_graph["Disease_id"] == dz_id]["Disease_id"].drop_duplicates().iloc[0]
#   gene_description = combination_graph[combination_graph["Compound_id"] == gene_id]["Compound_id"].drop_duplicates().iloc[0]
  dot_product = ranked_dot_products[curr_k]

#   print('edge=({},{}), \t dotprod={:.2f},\t descriptions=({},{},{},{})'.format(dz_i, gene_i, dot_product, dz_description, gene_description,dz_id,gene_id))
  print('edge=({},{}), \t dotprod={:.2f},\t descriptions=({},{})'.format(dz_i, gene_i, dot_product,dz_id,gene_id))

  curr_k += 1
  if curr_k > top_k:
    break


Top Predicted Edges
edge=(3826,9421), 	 dotprod=72.34,	 descriptions=(Disease::MESH:D018805,Compound::DB00058)
edge=(3826,9164), 	 dotprod=71.77,	 descriptions=(Disease::MESH:D018805,Compound::DB02659)
edge=(3826,16945), 	 dotprod=68.86,	 descriptions=(Disease::MESH:D018805,Compound::CHEMBL10971)
edge=(3826,25524), 	 dotprod=65.42,	 descriptions=(Disease::MESH:D018805,Compound::CHEMBL1723343)
edge=(3826,10135), 	 dotprod=62.96,	 descriptions=(Disease::MESH:D018805,Compound::MESH:C075631)
edge=(3826,5127), 	 dotprod=60.29,	 descriptions=(Disease::MESH:D018805,Compound::CHEMBL99369)
edge=(3826,21334), 	 dotprod=59.54,	 descriptions=(Disease::MESH:D018805,Compound::MESH:D002109)
edge=(3826,19638), 	 dotprod=56.84,	 descriptions=(Disease::MESH:D018805,Compound::DB06762)
edge=(3826,16764), 	 dotprod=56.42,	 descriptions=(Disease::MESH:D018805,Compound::DB07943)
edge=(3826,18976), 	 dotprod=54.41,	 descriptions=(Disease::MESH:D018805,Compound::DB08833)
edge=(3826,17543), 	 dotprod=52.72,	 d

In [154]:
check = combination_graph[(combination_graph['source_node'].str.contains('D018805')) | (combination_graph['destination_node'].str.contains('D018805'))]

In [158]:
check

,source_node,destination_node,category
1521973,Compound::DB01330,Disease::MESH:D018805,Compound:Disease
1616729,Compound::MESH:C102006,Disease::MESH:D018805,Compound:Disease
1616861,Compound::CHEBI:15372,Disease::MESH:D018805,Compound:Disease
1616979,Compound::DB01775,Disease::MESH:D018805,Compound:Disease
1617029,Compound::DB14546,Disease::MESH:D018805,Compound:Disease
...,...,...,...
1787517,Gene::93759,Disease::MESH:D018805,Gene:Disease
1788296,Gene::9518,Disease::MESH:D018805,Gene:Disease
1788363,Gene::953,Disease::MESH:D018805,Gene:Disease
1788513,Gene::958,Disease::MESH:D018805,Gene:Disease


In [157]:
check[(check['source_node'].str.contains('CHEBI:18186')) | (check['destination_node'].str.contains('CHEBI:18186'))]

,source_node,destination_node,category


In [141]:
# combination_graph[(combination_graph['Compound_id'] == 'Compound::DB06404') & (combination_graph['Disease_id']=='Compound::DB06404')]